In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import layers

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y_train = train['label']
X_train = train.drop(columns = 'label')

In [5]:
Y_train

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [6]:
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, test = X_train/255., test/255.

In [9]:
X_train.isnull().describe()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000,42000,42000,42000,42000,42000,42000,42000,42000,42000,...,42000,42000,42000,42000,42000,42000,42000,42000,42000,42000
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,42000,42000,42000,42000,42000,42000,42000,42000,42000,42000,...,42000,42000,42000,42000,42000,42000,42000,42000,42000,42000


In [10]:
test.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [11]:
X_train = X_train.values.astype(np.float32).reshape(-1,28,28,1)

In [12]:
X_train.shape

(42000, 28, 28, 1)

In [13]:
test = test.values.astype(np.float32).reshape(-1,28,28,1)

In [14]:
test.shape

(28000, 28, 28, 1)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

In [16]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)

In [17]:
X_val.shape

(8400, 28, 28, 1)

In [18]:
EPOCHS = 1 # 30, 50, 100
batch_size = 64
drop_out = 0.25

In [19]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        conv = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        do = tf.keras.layers.Dropout
        self.sequence = []
        self.sequence.append(conv(32,kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape = (28,28,1)))
        self.sequence.append(conv(32,kernel_size = (5,5), padding = 'same', activation = 'relu'))
        self.sequence.append(maxpool((2,2)))
        self.sequence.append(do(drop_out))
        
        self.sequence.append(conv(64,kernel_size = (5,5), padding = 'same', activation = 'relu'))
        self.sequence.append(conv(64,kernel_size = (5,5), padding = 'same', activation = 'relu'))
        self.sequence.append(maxpool((2,2)))
        self.sequence.append(do(drop_out))
        
        self.sequence.append(conv(128,kernel_size = (5,5), padding = 'same', activation = 'relu'))
        self.sequence.append(conv(128,kernel_size = (5,5), padding = 'same', activation = 'relu'))
        self.sequence.append(maxpool((2,2)))
        
        self.sequence.append(tf.keras.layers.Flatten())
        self.sequence.append(tf.keras.layers.Dense(512, activation = 'relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation = 'softmax'))
        
    def call(self, x):
        for layer in self.sequence:
            x = layer(x)
        
        return x

In [20]:
@tf.function
def train_step(model, image, label, loss_obj, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_obj(label, predictions)    
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(label, predictions)
    
@tf.function
def test_step(model, image, label, loss_obj, test_loss, test_accuracy):
    predictions = model(image)
    loss = loss_obj(label, predictions)
    test_loss(loss)
    test_accuracy(label, predictions)

In [21]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(batch_size)

In [22]:
# Create model
model = CNN()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [68]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.16630519926548004, Accuracy: 94.5999984741211, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 2, Loss: 0.05305822938680649, Accuracy: 98.45238494873047, Test Loss: 0.0, Test Accuracy: 0.0


KeyboardInterrupt: 

In [23]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

525/525 [==============================] - 170s 325ms/step - loss: 0.2698 - accuracy: 0.9113 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [71]:
samp = pd.read_csv('sample_submission.csv')
samp.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [25]:
len(test)

28000

In [26]:
results = model.predict(test)

#results = np.argmax(results,axis = 1)

#submission = pd.Dataframe({'ImageId' : range(1,len(test)+1), 'Label' : results})
#submission.to_csv("submission.csv", index = False)

In [27]:
results.shape

(28000, 10)

In [30]:
np.argmax(results, axis = 1)

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)